# Wrangling Report for WeRateDog Twitter Data

## Table of Content
- [Part I - Gather](#gather)
- [Part II - Access](#access)
- [Part III - Clean](#clean)

<a id='gather'></a>
## Part I - Gather
1. Manually download `twitter-archive-enhanced.csv` file and upload to the same directory of `wrangle_act.ipynb`
2. Download `image-predictions.tsv` file by `Requests` library
3. Obtain the JSON file of each tweet in `twitter-archive-enhanced.csv` file using Twitter's API `tweepy` and save all to `tweet-json.txt` file.
4. Extract *tweet_id*, *favorite_count*, *retweet_count* to `tweet-extended.csv` file

<a id='access'></a>
## Part II - Access
Access `twitter-archive-enhanced.csv`, `image-predictions.tsv`, `tweet-extended.csv` files and find out the following items to be fixed:

#### Quality issues:
- `twitter-archive-enhanced.csv` table contains 181 retweets.
- Some tweets are missing in `tweet-extended.csv` and `image-predictions.tsv` tables.
- Columns related to retweets are not necessary in the table: *retweeted_status_id*, *retweeted_status_user_id*, *retweeted_status_timestamp*
- Columns *in_reply_to_status_id*, *in_reply_to_user_id* are also irrelevant.
- Timestamp should be in datetime format.
- The denominator is not 10 for 23 rows.
- Some rows contain very high or very low numerators.
- Some rows may contain 2 different stages.
- One more column in `image-predictions.tsv` table is needed to contain the prediction result for each tweet for analysis.

#### Tidiness issues:
- Doggo, floofer, pupper, and puppo should be in one categorical column
- `tweet-extended.csv` and `twitter-archive-enhanced.csv` table should be combined because they describe one observation (tweet).

<a id='clean'></a>
## Part III - Clean
1. Remove the retweet by dropping rows with `retweet_status_id` value
2. Some tweet_id are deleted when I downloaded the tweet JSON files and the `image-predictions.tsv` table doesn't contain prediction for each tweet. So getting the common rows from 3 tables are the best solution. Pandas `merge` function is used here.
3. Drop the irrelevant columns: *retweeted_status_id*, *retweeted_status_user_id* and *retweeted_status_timestamp*, *in_reply_to_status_id* and *in_reply_to_user_id*
4. Split the merged table to two tables: `tweet_clean` and `image_clean`. `tweet_clean` contains information about tweets, while `image_clean` contains information about prediction of images.

#### For `tweet_clean` table
5. Check if any tweet contains two dog stage information and drop them.
6. Create a new column *none* for rows without stage information and convert *doggo*, *floofer*, *pupper*, *puppo*, *none* to one column *stage* and then set the column type to category
7. Convert the *timestamp* column to datetime
8. Investigate the 23 rows with non-10 denominator. Drop rows without rating information and set others' denominator to 10.
9. check if the number before '/10' matches with numerator. If not, correct it.

#### For `image_clean` table
The `image` table contains top 3 predictions of the picture given by neural network. I used the prediction result which is a dog breed and also has highest confidence as the result of prediction for each tweet and then catogery the tweets by the top 10 dog breeds, not_dog and other_dog.
10. Delete the tweet_id which exist in `image_clean` table but not in `tweet_clean` table. 
11. Create a new column `prediction` and the value is the most likely prediction results which is a dog breed
12. Create a new column `breed`, making the value the same as `prediction` value for top 10 dog breeds and not_dog and changing others to other_dog
13. Change the `breed` column to category

#### Final Cleanup
14. Reset index for two tables and check if two tables contain the same tweet_id

#### Data Storge
15. Save two tables to two files `tweet_clean.csv` and `image_clean.csv`.
16. Merge two tables and store it as one file `twitter_archive_master.csv`.